In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import csv
import os

In [2]:
class KaggleLoadSubmit:
    def __init__(self, competition_name, csv_dir):
        """Kaggleのデータのダウンロードおよび提出を行うクラス

        Args:
            competition_name (string): Kaggleのコンペ名
            csv_dir (string): ダウンロード先と、提出先のcsvのディレクトリをstringで指定
        """
        self.kaggle_api = KaggleApi()
        self.csv_dir = csv_dir
        self.competition_name = competition_name
        if self.csv_dir[-1] != '\\':
            self.csv_dir += '\\'
        self.kaggle_api.authenticate()

    def download_zip(self):
        """コンペティションのzipファイルをダウンロードする
        """
        ret = self.kaggle_api.competition_download_files(self.competition_name, self.csv_dir, force=True)
        print(ret)

    def submit_csv(self, csv_name, message):
        """csvデータを提出する

        Args:
            csv_name (string): csvの名前
            message (string): 提出メッセージ
        """
        csv_path = self.csv_dir + csv_name
        self.kaggle_api.competition_submit(csv_path, message, self.competition_name)

In [3]:
kls = KaggleLoadSubmit('titanic', 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic')
# kls.submit_csv('op.csv', 'hoge')

# Titanic
https://www.kaggle.com/c/titanic/

In [4]:
import numpy as np
import pandas as pd

In [5]:
# データの読み込み
data_dir = 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic\\data'
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [6]:
def fix(data: pd.DataFrame):
    data.Sex.replace(['male', 'female'], [0, 1], inplace=True)
    data.Embarked.fillna(('S'), inplace=True)
    data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
    data.Fare.fillna(np.mean(data.Fare), inplace=True)
    age_avg = data.Age.mean()
    age_std = data['Age'].std()
    data['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)
    delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
    data.drop(delete_columns, axis=1, inplace=True)

fix(train)
fix(test)


In [7]:
display(test)

,Pclass,Sex,Age,Fare,Embarked
0,3,0,34.5,7.8292,2
1,3,1,47.0,7.0000,0
2,2,0,62.0,9.6875,2
3,3,0,27.0,8.6625,0
4,3,1,22.0,12.2875,0
...,...,...,...,...,...
413,3,0,32.0,8.0500,0
414,1,1,39.0,108.9000,1
415,3,0,38.5,7.2500,0
416,3,0,32.0,8.0500,0


In [8]:
# ロジスティック回帰で予測する
from sklearn.linear_model import LogisticRegression

def predict():
    Y_train = train['Survived']
    X_train = train.drop('Survived', axis=1)
    X_test = test
    clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    return y_pred

y_pred = predict()


C:\Users\Takanori\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [9]:
# save_csv
sub = pd.read_csv(os.path.join(data_dir, 'gender_submission.csv'))
sub['Survived'] = y_pred
sub.to_csv(os.path.join(data_dir, 'my_submission.csv'), index=False)

In [10]:
kls.submit_csv(os.path.join('my_submission.csv'), '0314')

100%|██████████| 3.18k/3.18k [00:03<00:00, 872B/s]
